In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [2]:
website_url=requests.get('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup=BeautifulSoup(website_url,'lxml') # or use html.parser
# print(soup.prettify())

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'}).tbody

In [4]:
My_table

<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York

In [4]:
PC=[]
B=[]
NH=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        PC.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        NH.append(cells[2].find(text=True))
        
import pandas as pd
df=pd.DataFrame(PC,columns=['PostCode'])
df['Borough']=B
df['Neighborhood']=NH
df.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
#df.reset_index(inplace=True)
gb = df.groupby(['PostCode','Borough'])
gb

In [6]:
result = gb['Neighborhood'].unique()
#result

In [8]:
#df.to_csv(r'path'+ '\\Filename', index.false)

In [7]:
df2=pd.DataFrame(result)
df2.reset_index(inplace=True) 
df2.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,[Not assigned ]
1,M1B,Scarborough,"[Rouge, Malvern]"
2,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
3,M1E,Scarborough,"[Guildwood , Morningside, West Hill]"
4,M1G,Scarborough,[Woburn]


In [8]:
def unfold(obj):
    toReturn = []
    for row in obj:
        strHolder = ""
        for x in row:
            if np.where(row == x)[0] == 0:
                strHolder = strHolder + x.replace('\n', '')
            else:
                strHolder = strHolder + ', ' + x.replace('\n', '')
        toReturn.append(strHolder)
    return toReturn



In [9]:
#print(np.array2string(df2[0,2].replace(r"[",'').replace(r"]",'')))
df2['Neighborhood']=  unfold(df2['Neighborhood'])
df2.head()
# unfold(df2['Neighborhood'])

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [10]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_loc = pd.merge(left=df2,right=df_data_1, how='left', left_on='PostCode', right_on='Postal Code')
df_loc.head()

,PostCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN,NaN
1,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
4,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917


In [12]:
df_loc

,PostCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN,NaN
1,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
4,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
5,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
6,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476


In [13]:
!conda install -c conda-forge geocoder --yes
import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [ ]:
# Create a copy of the part_01 dataframe
df3 = df2.copy()

# Create a list of all the Postal Codes
postal_codes = df3['PostCode'].tolist()

# Search for the Latitude and Longitude of each postal code
for postal_code in postal_codes:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    print('{}, Toronto, Ontario'.format(postal_code))
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
# g = geocoder.opencage('{}, Toronto, Ontario'.format(postal_code), key='13aa077d21ae42f287ec8607e07b2159')

        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    df3.loc[df3.Postcode == postal_code, 'Latitude'] = latitude
    df3.loc[df3.Postcode == postal_code, 'Longitude'] = longitude

M1A, Toronto, Ontario


In [ ]:
df3.head()